In [8]:
import pandas as pd
from google.cloud import storage
from google.api_core import page_iterator
from datetime import datetime, date, timedelta
import os

# Utility Function to List Out Google Clou Storage Directories for a Given File Path

In [9]:
def _item_to_value(iterator, item):
    return item

def list_directories(bucket_name, prefix):
    if prefix and not prefix.endswith('/'):
        prefix += '/'

    extra_params = {
        "projection": "noAcl",
        "prefix": prefix,
        "delimiter": '/'
    }

    gcs = storage.Client()

    path = "/b/" + bucket_name + "/o"

    iterator = page_iterator.HTTPIterator(
        client=gcs,
        api_request=gcs._connection.api_request,
        path=path,
        items_key='prefixes',
        item_to_value=_item_to_value,
        extra_params=extra_params,
    )

    return [x for x in iterator]

# Define Query to Execute Against GBQ for Keeper Txn Time Calculation

In [16]:
query = '''

with
base as (
  select
    ttl.entropy_keeper_address,
    case when ttl.instruction_type = 'ConsumeEvents' then ttl.instruction_type else 'OtherEvents' end as instruction_type,
    sum(coalesce(ttl.seconds_since_last_txn,0)) as total_time_by_address_instruction_type
  from entropy.txns_time_lag ttl
  where
    date(ttl.date_time) = date('{}')
  group by 1,2
)

select
  b.entropy_keeper_address,
  b.instruction_type,
  b.total_time_by_address_instruction_type,
  sum(b.total_time_by_address_instruction_type) over (partition by instruction_type) as total_time_overall,
  (b.total_time_by_address_instruction_type * 1.0) / sum(b.total_time_by_address_instruction_type) over (partition by instruction_type) as pct_of_time
from base b

'''

# Initialize Google Cloud Storage Client and Resources

In [58]:
client = storage.Client()
txn_bucket = client.get_bucket('entropy-keeper-transactions')
rewards_bucket = client.get_bucket('entropy-rewards')
rewards_rate_table = pd.read_parquet('gs://entropy-rewards/program-references/2022-05-01-rewards-rate-table.parquet')
rewards_rate_table['date'] = rewards_rate_table['date'].apply(lambda x: str(x))
rewards_rate_table.set_index('date',inplace=True)

In [36]:
date_list = [x.strip("'raw/").strip('/') for x in list_directories('entropy-keeper-transactions', 'raw/')]
date_list[:-1]

['2022-04-22',
 '2022-04-23',
 '2022-04-24',
 '2022-04-25',
 '2022-04-26',
 '2022-04-27',
 '2022-04-28',
 '2022-04-29',
 '2022-04-30',
 '2022-05-01']

# Loop Through Txns and Aggregate Rewards Per Keeper Wallet

In [68]:
for date in date_list[:-1]:
    print(datetime.now(), "Starting the process for the {} txn set...".format(date))

    rewards_df = pd.read_gbq(query=query.format(date))

    rewards_df['daily_total_reward'] = rewards_df['instruction_type'].apply(
        lambda x: rewards_rate_table.loc[date]['consume_events_reward'] if x == 'ConsumeEvents' else rewards_rate_table.loc[date]['other_events_reward']
    )

    rewards_df['daily_keeper_reward'] = rewards_df['pct_of_time'] * rewards_df['daily_total_reward']
    rewards_df['date'] = date
    
    print(datetime.now(), 'Creating parquet file...')   
    rewards_df.to_parquet(date+'-daily-rewards.parquet')
    
    print(datetime.now(), 'Writing data to GBQ...')
    rewards_df.to_gbq('entropy.keeper_rewards_daily',if_exists='append')

    print(datetime.now(), 'Uploading file to GCS...')
    blob = rewards_bucket.blob('daily/'+date[:7]+'/'+date+'-daily-rewards.parquet')
    blob.upload_from_filename(date+'-daily-rewards.parquet')

    print(datetime.now(), 'Deleting file from local memory...')
    os.remove(date+'-daily-rewards.parquet')

2022-05-02 09:18:05.407512 Starting the process for the 2022-04-22 txn set...
2022-05-02 09:18:06.360365 Creating parquet file...
2022-05-02 09:18:06.363582 Writing data to GBQ...


100%|██████████| 1/1 [00:00<00:00, 10699.76it/s]


2022-05-02 09:18:11.498162 Uploading file to GCS...
2022-05-02 09:18:11.735836 Deleting file from local memory...
2022-05-02 09:18:11.736367 Starting the process for the 2022-04-23 txn set...
2022-05-02 09:18:12.638360 Creating parquet file...
2022-05-02 09:18:12.641043 Writing data to GBQ...


100%|██████████| 1/1 [00:00<00:00, 14169.95it/s]


2022-05-02 09:18:16.966567 Uploading file to GCS...
2022-05-02 09:18:17.227068 Deleting file from local memory...
2022-05-02 09:18:17.227718 Starting the process for the 2022-04-24 txn set...
2022-05-02 09:18:18.070055 Creating parquet file...
2022-05-02 09:18:18.072998 Writing data to GBQ...


...

2022-05-02 09:18:53.109135 Uploading file to GCS...
2022-05-02 09:18:53.336255 Deleting file from local memory...
2022-05-02 09:18:53.336853 Starting the process for the 2022-04-30 txn set...
2022-05-02 09:18:54.111631 Creating parquet file...
2022-05-02 09:18:54.114472 Writing data to GBQ...


100%|██████████| 1/1 [00:00<00:00, 14266.34it/s]


2022-05-02 09:19:00.156361 Uploading file to GCS...
2022-05-02 09:19:00.390249 Deleting file from local memory...
2022-05-02 09:19:00.390786 Starting the process for the 2022-05-01 txn set...
2022-05-02 09:19:01.729228 Creating parquet file...
2022-05-02 09:19:01.732009 Writing data to GBQ...


100%|██████████| 1/1 [00:00<00:00, 12905.55it/s]

2022-05-02 09:19:04.359476 Uploading file to GCS...
2022-05-02 09:19:04.517041 Deleting file from local memory...
